# Custom Chatbot Project

In this project I'm going to use the **2023 Fashion Trends** dataset, which contains information about recent fashion developments and styles. This dataset is appropriate for demonstrating a chatbot that can answer questions about fashion trends, designers, and style changes in 2023.

## Data Wrangling

In [1]:
# Some imports required for this notebook
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv

from utils import *

In [2]:
# Set openai api key by providing a valid key or text file containing the key.
load_dotenv(".config.env")
OPENAI_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_URL = os.getenv('OPENAI_BASE_URL')
set_api_key(OPENAI_KEY, OPENAI_URL)

# Create a configuration object
config = create_config({
    'EMBEDDING_MODEL_NAME': 'text-embedding-ada-002',
    'COMPLETION_MODEL_NAME': 'gpt-4o-mini',
    'ENCODING': 'cl100k_base',
    'MAX_PROMPT_TOKENS': 2000,
    'MAX_RESPONSE_TOKENS': 150,
    'BATCH_SIZE': 100,
    'FROM_SCRATCH': True
})

In [3]:
if config.FROM_SCRATCH:
    df = pd.read_csv('./data/source/2023_fashion_trends.csv')
    df = clean_csv_data(df)
else:
    # Load cleaned data from csv file
    df = pd.read_csv('./data/results/df_preprocessed.csv', index_col = 0)

In [4]:
# EMBEDDINGS 

if config.FROM_SCRATCH:
    # Get embeddings for all text rows from openai and store in csv file
    df['embeddings'] = get_embeddings(df, config)
    df.to_csv('./data/results/df_embeddings.csv', index=False)
    df['embeddings'] = df['embeddings'].apply(np.array)
else:
    # Load preprocessed date with embeddings from csv file
    df = pd.read_csv('./data/results/df_embeddings.csv', index_col = 0)
    df['embeddings'] = df['embeddings'].apply(eval).apply(np.array)

In [5]:
df.head()

,text,embeddings
0,2023 Fashion Trend: Red.,"[-0.01590677909553051, -0.028028665110468864, ..."
1,Glossy red hues took over the Fall 2023 runway...,"[-0.018273593857884407, -0.022659776732325554,..."
2,Think: Juicy reds with vibrant orange underton...,"[-0.02649392932653427, -0.017718786373734474, ..."
3,7 Fashion Trends That Will Take Over 2023 — Sh...,"[-0.008027786388993263, -0.010925652459263802,..."
4,2023 Fashion Trend: Cargo Pants.,"[-0.006155569106340408, -0.02691546455025673, ..."


## Custom Query Completion

In [6]:
answer = answer_question("What are the top fashion trends in 2023?", df, config, custom=True)
print(answer)

The top fashion trends in 2023 include:

1. Red
2. Denim Reimagined
3. Cobalt Blue
4. Sheer Clothing
5. Maxi Skirts
6. Shine For The Daytime
7. Cargo Pants
8. Utilitarian wear
9. Naked dressing
10. Pared-back approaches to wardrobe
11. Neo-minimalism
12. 3D designs and floral motifs
13. Canadian tuxedo revival
14. Glossy red hues
15. Slinky, draped dresses and skirts.


In [7]:
# Example question and answer
answer = answer_question("Which designer influenced the 2023 fashion trends the most?", df, config, custom=True)
print(answer)

I don't know.


## Custom Performance Demonstration


### Question 1

In [8]:
# Question with context
print(answer_question('What colors are popular in 2023 fashion trends?', df, config, custom=True))

Popular colors in the 2023 fashion trends include red, cobalt blue, and various bold colors such as lilac, pale yellow, baby blue, and ice-cream shades. Additionally, shades of saffron and colors beyond khaki and olive are noted as well.


In [9]:
# Same question without context
print(answer_question('What colors are popular in 2023 fashion trends?', df, config, custom=False))

I don't know.


### Question 2

In [10]:
print(answer_question('Which materials are most used in 2023 fashion collections?', df, config, custom=True))

Based on the context, notable materials used in 2023 fashion collections include silk, organza, leather, thicker fabrications, PVC, and various printed fabrics. Additionally, there is an exploration of innovative materials like Fabrica and a focus on shiny metallics.


In [11]:
print(answer_question('Which materials are most used in 2023 fashion collections?', df, config, custom=False))

I don't know.


## Chat Bot

In [15]:
print('Hello, what do you want to know?\n')
while True:
    question = input('You: ')
    if len(question) > 0:
        print(f"User: {question}")
        print(f'Bot: {answer_question(question, df, config, custom=True)}', end='\n')
    else:
        print('\nGood bye!')
        break

Hello, what do you want to know?

User: What are trends of Fringe in fashion trend in 2023?
Bot: Fringe has long been associated with boho, western-influenced style, but it’s going in a new direction this spring. The context does not provide specific details about the new direction for fringe in 2023 fashion trends.
User: Are there any fashion trends about '90s in 2023?
Bot: Yes, there are several fashion trends in 2023 that are inspired by the '90s, including denim maxi skirts, bomber jackets, biker jackets, and ballet shoes. Additionally, there is a general return of '90s influences, such as grunge and minimalism.
User: What are fashion trend about sports in 2023?
Bot: Sport holds a huge influence over fashion, exemplified by items like the tennis skirt and cricket jumper, which confirms that sports trends are integrated into fashion. However, the specific fashion trends related to sports in 2023 are not detailed in the provided context. Therefore, I don't know.

Good bye!
